In [2]:
import pandas as pd

path = r'C:\Users\juan.sossa\projects_integrait\DNDI\data\db3bea42-add2-4185-9a08-d8a8a2622277.xlsx'
df_root = pd.read_excel(path)

In [2]:
df= df_root[df_root['name']=='Child Bearing Potential']
lista_sujetos = df['Participante'].unique()
df = df[['name', 'Visit', 'activityState', 'Participante', 'Estado del Participante', 'Campo', 'Valor']]

In [3]:
df_visit_date = df_root[df_root['name']=='Date of visit']
df_visit_date = df_visit_date[['Visit','Participante', 'Campo', 'Valor']]
df_visit_date = df_visit_date[df_visit_date['Campo']=='Visit Date']
df_visit_date = df_visit_date[['Visit','Participante','Valor']]
df_visit_date = df_visit_date.rename(columns={'Participante':'Subject'})

In [3]:
df_child_bearing  = df_root[df_root['name']=='Child Bearing Potential']
df_child_bearing = df_child_bearing[['Visit','Participante', 'Campo', 'Valor']]
df_child_bearing = df_child_bearing[df_child_bearing['Campo']=='Is the participant postmenopausal?']
df_child_bearing = df_child_bearing[['Visit','Participante','Valor']]
df_child_bearing = df_child_bearing.rename(columns={'Participante':'Subject', 'Valor': 'post_menopausal'})

In [4]:
df_child_bearing

,Visit,Subject,post_menopausal
1213,Screening Visit,011002,0
1214,Screening Visit,011008,1
1215,Screening Visit,011005,1
1216,Screening Visit,011007,1
1217,Screening Visit,012009,1
1218,Screening Visit,012010,0
1219,Screening Visit,012011,0
1220,Screening Visit,012012,0
1221,Screening Visit,012013,1
1222,Screening Visit,012014,1


In [4]:
df_demographic = df_root[df_root['name']=='Demographics']
df_demographic = df_demographic[['Visit','Participante', 'Campo', 'Valor']]
df_demographic = df_demographic[df_demographic['Campo']=='Gender']
df_demographic = df_demographic[['Visit','Participante','Valor']]
df_demographic = df_demographic.rename(columns={'Participante':'Subject', 'Valor':'Genero'})

In [12]:
df_demographic_age = df_root[df_root['name']=='Demographics']
df_demographic_age = df_demographic_age[['Visit','Participante', 'Campo', 'Valor']]
df_demographic_age = df_demographic_age[df_demographic_age['Campo']=='Birth Year']
df_demographic_age = df_demographic_age[['Visit','Participante','Valor']]
df_demographic_age = df_demographic_age.rename(columns={'Participante':'Subject', 'Valor':'Genero'})

In [10]:
df_demographic

,Visit,Subject,Genero
1165,Screening Visit,011002,1
1166,Screening Visit,011003,2
1167,Screening Visit,011004,1
1168,Screening Visit,011008,1
1169,Screening Visit,011005,1
1170,Screening Visit,011006,2
1171,Screening Visit,011007,1
1172,Screening Visit,012009,1
1173,Screening Visit,012010,1
1174,Screening Visit,012011,1


# 1 -> Males
# 2 -> Female

In [5]:
df_demographic.head(3)

,Visit,Subject,Genero
1165,Screening Visit,011002,1
1166,Screening Visit,011003,2
1167,Screening Visit,011004,1


In [31]:
prueba = df[['Campo', 'Valor']].T
columns_analize = prueba.iloc[0].unique()
columns_analize

array(['Is the participant postmenopausal?', 'Month of Last Menstruation',
       'Year of Last Menstruation',
       'Is the FSH test result available and ≥ 40 IU/L?',
       'Contraception of non post-menopausal woman',
       'Use of combined (estrogen and progestogen-containing) hormonal contraception. associated with inhibition of ovulation',
       'Use of progestogen-only hormonal contraception',
       'Date of start of contraceptive method',
       'Date of start of systematic use of condom', 'AND'], dtype=object)

In [9]:
from datetime import datetime
from revision_fechas import revision_fecha
import warnings

warnings.filterwarnings('ignore')

lista_revision = []

# fecha_inicio = datetime.strptime('19-06-2023', "%d-%m-%Y")
# fecha_fin =  datetime.strptime('31-10-2023', "%d-%m-%Y")

for sujeto in lista_sujetos:
    sujeto_principal = df[df['Participante']==sujeto]

    for visita in sujeto_principal.Visit.unique():
        pru_1 = sujeto_principal[sujeto_principal['Visit']==visita]
        pru = pru_1
        pru = pru[['Campo', 'Valor']].T
        new_columns = pru.iloc[0]
        pru = pru[1:].set_axis(new_columns, axis=1)
        pru['Subject'] = sujeto
        pru['Visit'] = visita
        pru['status'] = pru_1['activityState'].unique()
        pru = pru.merge(df_visit_date, on=['Subject', 'Visit'], how='left')
        pru = pru.merge(df_demographic, on=['Subject', 'Visit'], how='left')
        pru = pru.merge(df_demographic_age, on=['Subject', 'Visit'], how='left')

        for index, row in pru.iterrows():
            status = row['status']
            if status == 'INCOMPLETE' or status == 'PENDING_QUERIES':
                pass
            else:
                subject = row['Subject']
                visit = row['Visit']
                fecha_visita = row['Valor']
                genero = float(row['Genero'])
                if genero == 1.0:
                    try:
                        participant_postmenopausical = row['Is the participant postmenopausal?']
                        if str(participant_postmenopausical) != '' or  participant_postmenopausical != float('nan') or str(participant_postmenopausical) != 'N/A':
                            error = [subject, visit, 'If Subjects Gender is "Male" in DEMOGRAPHIC, form should be left empty' , 'Form isnt empty', 'CB0010']
                            lista_revision.append(error)
                    except:
                        pass

                    try:
                        last_mestruation_month = row['Month of Last Menstruation']
                        if str(last_mestruation_month) != '' or  last_mestruation_month != float('nan') or str(last_mestruation_month) != 'N/A':
                            error = [subject, visit, 'If Subjects Gender is "Male" in DEMOGRAPHIC, form should be left empty' , 'Form isnt empty', 'CB0010']
                            lista_revision.append(error)
                    except:
                        pass

                    try:
                        last_mestruation_year = row['Year of Last Menstruation']
                        if str(last_mestruation_year) != '' or  last_mestruation_year != float('nan') or str(last_mestruation_year) != 'N/A':
                            error = [subject, visit, 'If Subjects Gender is "Male" in DEMOGRAPHIC, form should be left empty' , 'Form isnt empty', 'CB0010']
                            lista_revision.append(error)
                    except:
                        pass
                    try:
                        fsh_available = row['Is the FSH test result available and ≥ 40 IU/L?']
                        if str(fsh_available) != '' or  fsh_available != float('nan') or str(fsh_available) != 'N/A':
                            error = [subject, visit, 'If Subjects Gender is "Male" in DEMOGRAPHIC, form should be left empty' , 'Form isnt empty', 'CB0010']
                            lista_revision.append(error)
                    except:
                        pass
                    try:
                        contraception = row['Contraception of non post-menopausal woman']
                        if str(contraception) != '' or  contraception != float('nan') or str(contraception) != 'N/A':
                            error = [subject, visit, 'If Subjects Gender is "Male" in DEMOGRAPHIC, form should be left empty' , 'Form isnt empty', 'CB0010']
                            lista_revision.append(error)
                    except: 
                        pass
                    try:
                        use_combined_hormonal = row['Use of combined (estrogen and progestogen-containing) hormonal contraception. associated with inhibition of ovulation']
                        if str(use_combined_hormonal) != '' or  use_combined_hormonal != float('nan') or str(use_combined_hormonal) != 'N/A':
                            error = [subject, visit, 'If Subjects Gender is "Male" in DEMOGRAPHIC, form should be left empty' , 'Form isnt empty', 'CB0010']
                            lista_revision.append(error)
                    except:
                        pass
                    try:
                        progeston_hormonal = row['Use of progestogen-only hormonal contraception']
                        if str(progeston_hormonal) != '' or  progeston_hormonal != float('nan') or str(progeston_hormonal) != 'N/A':
                            error = [subject, visit, 'If Subjects Gender is "Male" in DEMOGRAPHIC, form should be left empty' , 'Form isnt empty', 'CB0010']
                            lista_revision.append(error)
                    except:
                        pass
                    try:
                        date_start_contraceptive = row['Date of start of contraceptive method']
                        if str(date_start_contraceptive) != '' or  date_start_contraceptive != float('nan') or str(date_start_contraceptive) != 'N/A':
                            error = [subject, visit, 'If Subjects Gender is "Male" in DEMOGRAPHIC, form should be left empty' , 'Form isnt empty', 'CB0010']
                            lista_revision.append(error)
                    except:
                        pass
                    try:
                        date_of_start_condom = row['Date of start of systematic use of condom']
                        if str(date_of_start_condom) != '' or  date_of_start_condom != float('nan') or str(date_of_start_condom) != 'N/A':
                            error = [subject, visit, 'If Subjects Gender is "Male" in DEMOGRAPHIC, form should be left empty' , 'Form isnt empty', 'CB0010']
                            lista_revision.append(error)
                    except:
                        pass
                         
                else:

                    try:
                        participant_postmenopausical = row['Is the participant postmenopausal?'] 
                    except:
                        pass
                    try:
                        last_mestruation_month = row['Month of Last Menstruation']
                    except:
                        pass
                    #try:
                    vist_date_year =  fecha_visita.split('-')[2]
                    last_mestruation_year = row['Year of Last Menstruation']
                    if int(vist_date_year) - int(last_mestruation_year) >= 1:
                        pass
                    else:
                        print('estamalo esto')
                    # except:
                    #     pass
                    try:
                        fsh_available = row['Is the FSH test result available and ≥ 40 IU/L?']
                    except:
                        pass
                    try:
                        contraception = row['Contraception of non post-menopausal woman']
                    except:
                        pass
                    try:
                        use_combined_hormonal = row['Use of combined (estrogen and progestogen-containing) hormonal contraception. associated with inhibition of ovulation']
                    except:
                        pass
                    try:
                        progeston_hormonal = row['Use of progestogen-only hormonal contraception']
                    except:
                        pass
                    try:
                        date_start_contraceptive = row['Date of start of contraceptive method']
                        
                    except:
                        pass
                    try:
                        date_of_start_condom = row['Date of start of systematic use of condom']
                      
                    except:
                        pass

IndexError: list index out of range

In [35]:
lista_revision

TypeError: unhashable type: 'list'